In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("tested.csv")

In [3]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892.0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,330911,211.3375,NaN,S
1,893.0,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,363272,151.5500,NaN,S
2,894.0,0,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,240276,151.5500,NaN,S
3,895.0,0,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,315154,151.5500,NaN,S
4,896.0,0,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,3101298,151.5500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,NaN,0,3,"Zabour, Miss. Hileni",female,14.5000,1,0,NaN,14.4542,NaN,C
1305,NaN,0,3,"Zabour, Miss. Thamine",female,NaN,1,0,NaN,14.4542,NaN,C
1306,NaN,0,3,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,NaN,7.2250,NaN,C
1307,NaN,0,3,"Zakarian, Mr. Ortin",male,27.0000,0,0,NaN,7.2250,NaN,C


In [4]:
df.isnull().sum()

,0
PassengerId,891
Survived,0
Pclass,0
Name,0
Sex,0
Age,263
SibSp,0
Parch,0
Ticket,891
Fare,1


In [5]:
df.drop(columns='Cabin',inplace=True)

In [6]:
df['Fare'].fillna(df['Fare'].mean(),inplace=True)

In [7]:
df.corr(numeric_only=True)['Age'].abs()

,Age
PassengerId,0.178348
Survived,0.055513
Pclass,0.408106
Age,1.000000
SibSp,0.243699
Parch,0.150917
Fare,0.178229


In [8]:
df.groupby(['Sex','Pclass'])['Age'].median(numeric_only=True)

Sex     Pclass
female  1         36.0
        2         28.0
        3         22.0
male    1         42.0
        2         29.5
        3         25.0
Name: Age, dtype: float64

In [9]:
df['Age'] = df.groupby(['Sex','Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))

In [10]:
df['Title'] = df['Name'].str.split(", ",expand=True)[1].str.split(".",expand=True)[0]

In [11]:
df['Title'].value_counts()

,count
Title,
Mr,757
Miss,260
Mrs,197
Master,61
Rev,8
Dr,8
Col,4
Mlle,2
Ms,2


In [12]:
df['Title'] = df['Title'].replace(['Lady', 'the Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')

In [13]:
def Family_size(x):
  if x ==1:
    return "Alone"
  elif x>1 and x<4:
    return "Small"
  else:
    return "Large"

In [14]:
df['Family_size']=df['SibSp'] + df['Parch'] + 1
df['Family_size']=df['Family_size'].apply(Family_size)

In [15]:
df.drop(columns=['Name','Parch','SibSp','Ticket'],inplace=True)

In [16]:
df['Family_size']

,Family_size
0,Alone
1,Large
2,Large
3,Large
4,Large
...,...
1304,Small
1305,Small
1306,Alone
1307,Alone


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    float64
 1   Survived     1309 non-null   int64  
 2   Pclass       1309 non-null   int64  
 3   Sex          1309 non-null   object 
 4   Age          1309 non-null   float64
 5   Fare         1309 non-null   float64
 6   Embarked     1307 non-null   object 
 7   Title        1309 non-null   object 
 8   Family_size  1309 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 92.2+ KB


In [18]:
col=['Embarked','Pclass','Sex','Title','Family_size']
for i in col:
    print('Survival Correlation by:', i)
    df2=df.groupby(i)['Survived'].mean().reset_index()
    print(df2)
    print('-'*10, '\n')

Survival Correlation by: Embarked
  Embarked  Survived
0        C  0.555556
1        Q  0.357724
2        S  0.332604
---------- 

Survival Correlation by: Pclass
   Pclass  Survived
0       1  0.619195
1       2  0.429603
2       3  0.255289
---------- 

Survival Correlation by: Sex
      Sex  Survived
0  female  0.727468
1    male  0.190985
---------- 

Survival Correlation by: Title
    Title  Survived
0  Master  0.508197
1    Miss  0.678030
2      Mr  0.162483
3     Mrs  0.787879
4    Rare  0.379310
---------- 

Survival Correlation by: Family_size
  Family_size  Survived
0       Alone  0.302532
1       Large  0.360000
2       Small  0.548223
---------- 



In [19]:
df['Embarked'].fillna('S',inplace=True)

In [20]:
import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure()

fig.add_trace(go.Histogram(x=df[df['Survived']==0]['Age'], name='Not Survived', opacity=0.5))
fig.add_trace(go.Histogram(x=df[df['Survived']==1]['Age'], name='Survived', opacity=0.5))

fig.update_layout(
    title='Age Distribution by Survival',
    xaxis_title='Age',
    yaxis_title='Density',
    barmode='overlay',
    bargap=0.1,
)

fig.show()

In [21]:
import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure()

fig.add_trace(go.Histogram(x=df[df['Survived']==0]['Fare'], name='Not Survived', opacity=0.5))
fig.add_trace(go.Histogram(x=df[df['Survived']==1]['Fare'], name='Survived', opacity=0.5))

fig.update_layout(
    title='Age Distribution by Survival',
    xaxis_title='Age',
    yaxis_title='Density',
    barmode='overlay',
    bargap=0.1,
)

fig.show()

In [22]:
import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure()

fig.add_trace(go.Histogram(x=df[df['Survived']==0]['Family_size'], name='Not Survived', opacity=0.5))
fig.add_trace(go.Histogram(x=df[df['Survived']==1]['Family_size'], name='Survived', opacity=0.5))

fig.update_layout(
    title='Age Distribution by Survival',
    xaxis_title='Age',
    yaxis_title='Density',
    barmode='overlay',
    bargap=0.1,
)

fig.show()

In [23]:
from plotly.subplots import make_subplots
col = ['Embarked', 'Pclass', 'Sex', 'Title']

fig = make_subplots(rows=1, cols=len(col), subplot_titles=col)

for i, c in enumerate(col, start=1):
    grouped_data = df.groupby([c, 'Survived']).size().reset_index(name='Count')

    survived_data = grouped_data[grouped_data['Survived'] == 1]
    not_survived_data = grouped_data[grouped_data['Survived'] == 0]

    fig.add_trace(go.Bar(x=survived_data[c], y=survived_data['Count'], name='Survived', marker_color='green'), row=1, col=i)
    fig.add_trace(go.Bar(x=not_survived_data[c], y=not_survived_data['Count'], name='Not Survived', marker_color='red'), row=1, col=i)

    fig.update_xaxes(title_text=c, row=1, col=i)
    fig.update_yaxes(title_text='Count', row=1, col=i)

# Update figure layout
fig.update_layout(
    title='Countplot of Categorical Variables by Survival',
    height=400,
    showlegend=True,
    barmode='group',
)

# Show figure
fig.show()

In [24]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn import ensemble
from sklearn import gaussian_process
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import svm
from sklearn import tree
from sklearn import discriminant_analysis
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [25]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame and 'Survived' is the target column
X = df.drop(columns=['Survived','PassengerId'])  # Features
y = df['Survived']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting sets
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Labels Shape:", y_train.shape)
print("Testing Labels Shape:", y_test.shape)

Training Features Shape: (1047, 7)
Testing Features Shape: (262, 7)
Training Labels Shape: (1047,)
Testing Labels Shape: (262,)


In [26]:
X_test

,Pclass,Sex,Age,Fare,Embarked,Title,Family_size
1148,3,male,35.0,7.1250,S,Mr,Alone
1049,3,male,20.0,15.7417,C,Mr,Small
982,3,male,25.0,7.8958,S,Mr,Alone
808,3,male,25.0,8.0500,S,Mr,Alone
1195,3,male,25.0,7.7500,Q,Mr,Alone
...,...,...,...,...,...,...,...
572,2,female,28.0,12.6500,S,Mrs,Alone
140,1,male,23.0,63.3583,C,Mr,Small
1182,3,female,21.0,7.6500,S,Miss,Alone
312,1,male,50.0,211.5000,C,Mr,Small


In [27]:
num_cat_transformer = ColumnTransformer([
    ('scaling',MinMaxScaler(),[3,2]),
    ('ohe1',OneHotEncoder(),[0,4]),
    ('ohe2',OneHotEncoder(),[5,6]),
    ('ordinal',OrdinalEncoder(),[1])
],remainder='passthrough')

In [28]:
bins = ColumnTransformer([
    ('Kbins',KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile'),[2,3])
],remainder='passthrough')

In [29]:
def create_pipeline(algo):
  return Pipeline([
      ('num_cat_transformer',num_cat_transformer),
      ('bins',bins),
      ('classifier',algo)
  ])

In [30]:
algorithms=[
    # Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    # Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),

    # GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),

    # Naive Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),

    # Nearest Neighbor
    neighbors.KNeighborsClassifier(),

    # SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),

    # Trees
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),

    # Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    # XGBoost
    XGBClassifier(),

    #
    LogisticRegression()
]

In [31]:
model_names = []
CV_accuracy = []
test_accuracy = []
for algo in algorithms:
    pipeline = create_pipeline(algo)
    scores = cross_val_score(pipeline, X_train, y_train, cv=5)  # CV on training data
    # pipeline.fit(X_train, y_train)  # Fit on full training data
    y_pred = pipeline.predict(X_test)  # Predict on test data
    # model_names.append(algo.__class__.__name__)
    CV_accuracy.append(scores.mean())
    # test_accuracy.append(accuracy_score(y_test, y_pred))
    # print(f"{algo.__class__.__name__} CV Accuracy: {scores.mean():.4f}")
    # print(f"{algo.__class__.__name__} Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print("-" * 50)


NotFittedError: Pipeline is not fitted yet.

In [ ]:
print("Survived Distribution:")
print(df['Survived'].value_counts(normalize=True))
print("\nSex vs Survived:")
print(pd.crosstab(df['Sex'], df['Survived'], normalize='index'))
print("\nTitle vs Survived:")
print(pd.crosstab(df['Title'], df['Survived'], normalize='index'))